# Reality Mining. Jupyter Notebook implementation


With scipy library realitymining mat file is red and saved in to data variable.
Since mat lab contains a lot of data, reading process take a lot of computer resources!

In [1]:
import scipy.io
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import math
from sklearn.neural_network import BernoulliRBM

data = scipy.io.loadmat("realitymining.mat")['s']

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
affilation = data['my_affil']
data_mat = data['data_mat']

# Restricted Boltzmann machine

### Data preprocessing.

Extract data_mat elements for each subject, categorize subjects to sloan and no sloan and save index of subjects who belongs 'sloan' category.

In [3]:
sloan_list = []
features_list = []
observation_days = 7
count = 0

for i in range(len(affilation[0])):
    if len(data_mat[0][i]) > 0:
        if len(data_mat[0][i][0]) >= observation_days and len(affilation[0][i]) > 0:
            features_list += [data_mat[0][i]]
            if affilation[0][i][0][0][0] == 'sloan' or affilation[0][i][0][0][0] == 'sloan_2':
                sloan_list += [count]
            count += 1

### Exclude NaN values.
Change all NaN values to value 4 in feature_list data.

In [4]:
for subject in range(len(features_list)):
    for hour in range(len(features_list[subject])):
        for element in range(len(features_list[subject][hour])):
            if math.isnan(features_list[subject][hour][element]):
                features_list[subject][hour][element] = 4

### Creating Features vector for training and testing.
<br>
__observation data__ -  value of how many days subject activity is stored. By default we have one week - 7 days.
<br>
__all_places__ - it is a list of all possible places in data_set :

 Value | Explanation
  -------------  | -------------
  0 | No signal
  1 | Home
  2 | Work
  3 | Elsewhere
  4 | Phone is off
<br>
__count__ - Current index of element position in features vector.
<br>
__features__ - 2-rd dimensional numpy N-darray for storing features for RBM training:
<br>
* 1-st dimension - Subject
* 2-nd dimension - Subject activity each hour of the week.



In [5]:
#  0 – no signal, 1 – home, 2 – work, 3 – elsewhere, 4 – phone is off

all_places = [0, 1, 2, 3, 4]
count = 0

features = np.zeros((len(features_list), observation_days * 24 * len(all_places)))

for subject in range(len(features_list)):
    for number in range(len(all_places)):
        for week in range(observation_days):
            for hours in range(24):
                if features_list[subject][hours][week] == all_places[number]:
                    features[subject][count] = 1
                count += 1
    count = 0

In [6]:
rbm_list = []  
sk_rbm = BernoulliRBM(n_components=10, verbose=True, learning_rate=0.1, n_iter=50)
sk_rbm.fit(features)
rbm_list = sk_rbm.transform(features)

[BernoulliRBM] Iteration 1, pseudo-likelihood = -525.89, time = 0.07s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -513.22, time = 0.04s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -386.14, time = 0.01s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -364.58, time = 0.01s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -384.30, time = 0.01s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -372.57, time = 0.01s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -364.58, time = 0.01s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -453.69, time = 0.01s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -385.44, time = 0.01s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -240.16, time = 0.01s
[BernoulliRBM] Iteration 11, pseudo-likelihood = -257.39, time = 0.01s
[BernoulliRBM] Iteration 12, pseudo-likelihood = -317.14, time = 0.01s
[BernoulliRBM] Iteration 13, pseudo-likelihood = -265.26, time = 0.01s
[BernoulliRBM] Iteration 14, pseudo-likelihood = -264.80, time = 0.01s
[BernoulliRBM] 

# Multi-layer perceptron with tensorflow

### Labels creation for Multi-layer perceptron
Each subject is classified to sloan or no_sloan.
<br>
If subject belongs to __sloan__ cateogy, his label value is equal __1__.
<br>
If subject belongs to __no sloan__ category, his label value is equal __0__.

In [8]:
labels = []

for i in range(len(features)):
    if i in sloan_list:
        labels += [1]
    else:
        labels += [0]
        
labels = np.array(labels)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(rbm_list, labels.ravel(), test_size=0.5, random_state=0)
# Parameters that define the MLP
n_inputs = len(X_train[0])
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 2
X = tf.placeholder(tf.float32, shape= (None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [10]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        # Number of inputs
        n_inputs = int(X.get_shape()[1])
        
        # This value is computed to randomly initialize the weights
        stddev = 2 / np.sqrt(n_inputs)
        # Weigths can be initialized in different ways
        # Here they are randomly initialized from a Normal distribution (mean=0,std as computed before)
        # Notice that weights are organized in a matrix (tensor) and its number is n_inputs*n_neurons
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        # The variable that will contain the weights is W
        W = tf.Variable(init, name="kernel")
        
        # The variable that will contain the bias is b  
        # and is initialized to zero
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        
        # As in the perceptron what the neurons do is multiply the weights by 
        # the input

        Z = tf.matmul(X, W) + b
        
        # What the activation function does is to "process" the result
        # of the multiplication of weights by inputs, and this is the output
        # of every neuron. 
    
        if activation is not None:
            return activation(Z)
        else:
            return Z

In [11]:
# The scope name for this MLP is "dnn"
with tf.name_scope("dnn"):
    
    # The first hidden layer is defined using the RELU activation function
    # It will contain n_hidden1=300 hidden neurons and therefore output
    # 300 values    
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)

    # The second hidden layer is also defined using the RELU activation function
    # It will contain n_hidden2=100 hidden neurons and therefore output
    # 100 values    
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    
    # The output layer does not use any activation function
    # it will output n_outputs=10 values since there are 10 classes in MNIST
    logits = neuron_layer(hidden2, n_outputs, name="outputs")

In [12]:
# scope name of the loss function is "loss"
with tf.name_scope("loss"):
    
    # The Loss function is defined. It outputs a loss value for each x
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
#     xentropy = tf.contrib.keras.backend.binary_crossentropy(output=n_inputs, target = n_inputs, from_logits=False)
    
    # The total loss is mean of the loss values 
    loss = tf.reduce_mean(xentropy, name="loss")

In [13]:
learning_rate = 0.01

with tf.name_scope("train"):
    # The plain GradientDescentOptimizer is chosen
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [14]:
with tf.name_scope("eval"):
    # A prediction is correct if it is among the k=1 most probable
    # classes predicted by the NN. Since k=1, it is only correct
    # if the prediction coincides with the true class.
    correct = tf.nn.in_top_k(logits, y, 1)
    
    # The accuracy is the mean of correct predictions
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [15]:
# Initialization of the computation graph
init = tf.global_variables_initializer()

# tensorflow allows to define a saver to store the model after learning
saver = tf.train.Saver()

# Number of epochs
n_epochs = 100

# Size of the batch used to update the gradient
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(70 // batch_size):
            
            #Function next_batch automatically select the batch
#             X_batch, y_batch = mnist.train.next_batch(batch_size)
            
            # Weights are learned using the current batch            
            sess.run(training_op, feed_dict={X: X_train, y: y_train})
            
        # Accuracies are computed in the training and validation sets    
        acc_train = accuracy.eval(feed_dict={X:  X_train, y: y_train})
        acc_val = accuracy.eval(feed_dict={X: X_test,
                                            y: y_test})
        print(epoch, "Train accuracy:", acc_train, "Val accuracy:", acc_val)

0 Train accuracy: 0.735294 Val accuracy: 0.657143
1 Train accuracy: 0.735294 Val accuracy: 0.685714
2 Train accuracy: 0.735294 Val accuracy: 0.685714
3 Train accuracy: 0.735294 Val accuracy: 0.685714
4 Train accuracy: 0.764706 Val accuracy: 0.685714
5 Train accuracy: 0.764706 Val accuracy: 0.685714
6 Train accuracy: 0.764706 Val accuracy: 0.685714
7 Train accuracy: 0.764706 Val accuracy: 0.685714
8 Train accuracy: 0.764706 Val accuracy: 0.657143
9 Train accuracy: 0.764706 Val accuracy: 0.657143
10 Train accuracy: 0.764706 Val accuracy: 0.657143
11 Train accuracy: 0.764706 Val accuracy: 0.657143
12 Train accuracy: 0.764706 Val accuracy: 0.657143
13 Train accuracy: 0.764706 Val accuracy: 0.685714
14 Train accuracy: 0.764706 Val accuracy: 0.685714
15 Train accuracy: 0.764706 Val accuracy: 0.685714
16 Train accuracy: 0.764706 Val accuracy: 0.685714
17 Train accuracy: 0.764706 Val accuracy: 0.685714
18 Train accuracy: 0.764706 Val accuracy: 0.685714
19 Train accuracy: 0.794118 Val accuracy: